# Trabajo Practico 2019 - Reciclaje en CABA

## Pesaje Recibido en Puntos Verdes (2015-2016) en kilogramos

### Primero importamos las librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Importamos reemplazando comas por puntos para poder los pesajes a tipo flotante. Primero elimino los 4 puntos excepcionales que hay.

In [2]:
pwd

'C:\\Users\\emi\\Desktop\\UTN 2019\\CIENCIA DE DATOS\\tp\\tp al 12-11\\reciclaje-master'

In [3]:
comas_por_puntos = lambda x : (x.replace('.', '').replace(',', '.'))

filename = r'pesaje-recibido-en-puntos-verdes-2015-2016.csv'
pesajes = pd.read_csv(filename, delimiter=';', decimal='.',
                      converters = {'VIDRIO':comas_por_puntos,
                                    'PAPEL_Y_CARTON':comas_por_puntos,
                                    'METAL':comas_por_puntos,
                                    'PLASTICO_PET':comas_por_puntos,
                                    'TELGOPOR':comas_por_puntos,
                                    'PLASTICOS_TODOS_LOS_DEMAS':comas_por_puntos,
                                    'TETRA_BRICK':comas_por_puntos,
                                    'PEQUENIOS_ELECTRODOMESTICOS':comas_por_puntos,
                                    'APARATOS_ELECTRONICOS_EN_DESUSO':comas_por_puntos,
                                    'AVUS':comas_por_puntos})

In [4]:
print(f'Pesajes tiene: {np.shape(pesajes)[0]} filas y {np.shape(pesajes)[0]} columnas')

pesajes.head(3)

Pesajes tiene: 3080 filas y 3080 columnas


,PUNTO,COMUNA,ANIO,MES,SEMANA,VIDRIO,PAPEL_Y_CARTON,METAL,PLASTICO_PET,TELGOPOR,PLASTICOS_TODOS_LOS_DEMAS,TETRA_BRICK,PEQUENIOS_ELECTRODOMESTICOS,APARATOS_ELECTRONICOS_EN_DESUSO,AVUS
0,ESTACION VILLA SOLDATI,COMUNA 8,2015,ABRIL,1,31.2,20.7,0.27,,0,13.41,0,,,
1,ESTACION VILLA SOLDATI,COMUNA 8,2015,ABRIL,2,2.231,44.878,0,,0,37.886,0.54,,,
2,ESTACION VILLA SOLDATI,COMUNA 8,2015,ABRIL,3,38.042,27.763,1.067,,0,17.57,0,,,


###  Mostramos los puntos y comunas distintas

In [5]:
comunas_distintas = np.unique(pesajes.COMUNA)
print(f'Hay {len(comunas_distintas)} comunas distintas')

puntos_distintos = np.unique(pesajes.PUNTO)
print(f'Hay {len(puntos_distintos)} puntos distintos distribuidos entre las comunas')

Hay 15 comunas distintas
Hay 32 puntos distintos distribuidos entre las comunas


### Transformamos los meses a valores numéricos y separamos los años

In [6]:
meses = ["ENERO","FEBRERO","MARZO","ABRIL","MAYO","JUNIO","JULIO","AGOSTO","SEPTIEMBRE","OCTUBRE","NOVIEMBRE","DICIEMBRE"]
for i in range (1,13):
    pesajes.loc[ pesajes['MES'] == meses[i-1], 'MES'] = i

# Separo 2015 y 2016
pesajes_2015 = pesajes.loc[pesajes['ANIO'] == 2015]
pesajes_2016 = pesajes.loc[pesajes['ANIO'] == 2016]

### Vemos que porcenaje de datos faltantes hay por columna

In [7]:
# Primero reemplazo las cadenas vacias y los espacios ' ' por valor NaN para poder contarlos mas facil
pesajes = pesajes.replace('', np.nan).replace(' ', np.nan)

total = pesajes.isnull().sum().sort_values(ascending=False)
percent = (pesajes.isnull().sum()/pesajes.isnull().count()).sort_values(ascending=False)
nulos = pd.concat([total, percent], axis=1, keys=['Total', 'Porcentaje'])
nulos

,Total,Porcentaje
AVUS,2993,0.971753
APARATOS_ELECTRONICOS_EN_DESUSO,2872,0.932468
PEQUENIOS_ELECTRODOMESTICOS,2872,0.932468
PLASTICO_PET,1284,0.416883
TETRA_BRICK,738,0.239610
TELGOPOR,371,0.120455
METAL,371,0.120455
PAPEL_Y_CARTON,371,0.120455
VIDRIO,370,0.120130
PLASTICOS_TODOS_LOS_DEMAS,369,0.119805


#### Decidimos eliminar la columna de AVUs, y las de eletrodomesticos por tener menos del 90% de los datos
La cantidad de Nan en PEQUENIOS_ELECTRODOMESTICOS, APARATOS_ELECTRONICOS_EN_DESUSO y AVUS tienen su sentido, de los 32 puntos verdes, 30 son puntos verdes comunes que juntan vidrio, papel, carton, metal, plastico, tetra brick y plastico, y hay 2 puntos verdes que ademas de juntar lo anteriormente mencionado, juntan PEQUENIOS_ELECTRODOMESTICOS, APARATOS_ELECTRONICOS_EN_DESUSO y AVUS. Para poder hacer una mejor comparacion entre los puntos verdes, vamos a considerar los 32 iguales, descartando estas 3 columnas.

In [8]:
pesajes = pesajes.iloc[:,0:12]

Los PLASTICO_PET son de los materiales mas comunmente reciclables y lamentablemente es la columna que peor condicion tiene. Además de los registros nulos que tiene, tambien tienen en su mayoria valores 0. Por lo tanto decidimos eliminarla también.

In [9]:
vaciosPET = np.shape(pesajes.loc[(pesajes['PLASTICO_PET'] == "0")])[0] + pesajes.isnull().sum()[8]
print("Valores vacios en PLASTICO_PET: " + str(vaciosPET))

Valores vacios en PLASTICO_PET: 2311


In [10]:
pesajes = pesajes.iloc[:,[0,1,2,3,4,5,6,7,9,10,11]]
pesajes.head(3)

,PUNTO,COMUNA,ANIO,MES,SEMANA,VIDRIO,PAPEL_Y_CARTON,METAL,TELGOPOR,PLASTICOS_TODOS_LOS_DEMAS,TETRA_BRICK
0,ESTACION VILLA SOLDATI,COMUNA 8,2015,4,1,31.2,20.7,0.27,0,13.41,0
1,ESTACION VILLA SOLDATI,COMUNA 8,2015,4,2,2.231,44.878,0,0,37.886,0.54
2,ESTACION VILLA SOLDATI,COMUNA 8,2015,4,3,38.042,27.763,1.067,0,17.57,0


In [11]:
#Separo 2015 y 2016
pesajes_2015 = pesajes.loc[pesajes['ANIO'] == 2015]
np.shape(pesajes_2015)

(1731, 11)

In [12]:
pesajes_2016 = pesajes.loc[pesajes['ANIO'] == 2016]
np.shape(pesajes_2016)

(1349, 11)

# Analisis de cantidad de datos en mes
Antes de analizar las comunas, analizaremos si todos los meses estan con datos, para saber si nuestro dataframe llega de enero 2015 a diciembre 2016, o si el GCBA no recaudo data en algun periodo en particular.

Empezamos analizando 2015

In [13]:
# Primero analizo si todos los meses tienen datos
# Creo un dataframe que tenga mes, anio y la cantidad de datos que contiene, para saber si hay meses faltantes
meses_2015 = pd.DataFrame({"ANIO":[],"MES":[],"N_DATOS":[]})
meses_2015
mesAnterior = 0 #lo arranco en un mes inexistente
semanas_del_Mes = 0

for indice in range(0, len(pesajes_2015)):
    mesActual = pesajes_2015.iloc[indice].MES
    if (mesAnterior == mesActual):
        semanas_del_Mes += 1
    else:
        mesAnterior = mesActual
        meses_2015 = meses_2015.append(
                {"N_DATOS": semanas_del_Mes,
                 "ANIO": 2015,
                 "MES": mesActual}, ignore_index=True)
        semanas_del_Mes = 1 #Reseteo el contador

meses_2015_tabla = meses_2015.groupby(["MES", "ANIO"])["N_DATOS"].count().sort_values(ascending=False)
meses_2015_tabla

MES   ANIO  
12.0  2015.0    32
11.0  2015.0    32
10.0  2015.0    32
9.0   2015.0    32
8.0   2015.0    32
7.0   2015.0    32
6.0   2015.0    32
5.0   2015.0    32
4.0   2015.0    32
3.0   2015.0    32
2.0   2015.0    32
1.0   2015.0    32
Name: N_DATOS, dtype: int64

En 2015, las 32 comunas tienen al menos algun dato cargado en todos los meses, por lo que lo podemos considerar que en todo 2015 GCBA recgio datos de todos los puntos verdes.

Ahora analizamos 2016.

In [14]:
# Primero analizo si todos los meses tienen datos
# Creo un dataframe que tenga mes, anio y la cantidad de datos que contiene, para saber si hay meses faltantes
meses_2016 = pd.DataFrame({"ANIO":[],"MES":[],"N_DATOS":[]})
meses_2016
mesAnterior = 0 #lo arranco en un mes inexistente
semanas_del_Mes = 0

for indice in range(0, len(pesajes_2016)):
    mesActual = pesajes_2016.iloc[indice].MES
    if (mesAnterior == mesActual):
        semanas_del_Mes += 1
    else:
        mesAnterior = mesActual
        meses_2016 = meses_2016.append(
                {"N_DATOS": semanas_del_Mes,
                 "ANIO": 2016,
                 "MES": mesActual}, ignore_index=True)
        semanas_del_Mes = 1 #Reseteo el contador

meses_2016_tabla = meses_2016.groupby(["MES", "ANIO"])["N_DATOS"].count().sort_values(ascending=False)
meses_2016_tabla

MES   ANIO  
9.0   2016.0    32
8.0   2016.0    32
7.0   2016.0    32
5.0   2016.0    32
4.0   2016.0    32
3.0   2016.0    32
2.0   2016.0    32
1.0   2016.0    32
6.0   2016.0    30
10.0  2016.0     4
12.0  2016.0     2
11.0  2016.0     2
Name: N_DATOS, dtype: int64

En el mes 10-2016, solo 4 puntos verdes de 32 tienen datos, y tanto 11-2016 como 12-2016, solo dos puntos verdes de 32 tienen datos. Ante este panorama, vamos a descartar los ultimos 3 meses de 2016, y vamos a hacer nuestro analisis de enero 2015 a septiembre 2016.

In [15]:
# Almacenamos los meses 10,11 y 12 de 2016 para las 2 comunas especiales (PLAZA ARENALES y PARQUE CENTENARIO)
pesajes_extra_Arenales_Centenario = pesajes.loc[((pesajes['ANIO'] == 2016) & (pesajes['MES'].isin([10,11,12])) & (pesajes['PUNTO'].isin(['PLAZA ARENALES','PARQUE CENTENARIO'])))]
pesajes_extra_Arenales_Centenario.head()

# Eliminamos los ultimos 3 meses
pesajes = pesajes.loc[~((pesajes['ANIO'] == 2016) & (pesajes['MES'].isin([10,11,12])))]

##   Criterios elegidos para limpiar el Dataframe

Dada la gran cantidad de datos faltantes o vacios, expresados de diversas formas, elegimos un criterio para seleccionar los puntos mas prometedores para trabajar.
### 1) Semana
    Si una semana esta llena de 0s, "", " ", NaNs es considerada VACIA
### 2) Mes
    Si un mes tiene 3 o más semanas VACIAS -> Esta VACIO
    Si un mes no esta cargado en el dataframe -> Esta VACIO
### 3) Punto
    Si un Punto tiene 4 o mas meses VACIOS en 2015 y 2016 -> Esta VACIO



### Cuento los meses que aparecen tienen 3 o mas semanas vacias

In [16]:
#Primero filtro las que tienen meses con todas las filas llenas de cero (si tienen esas 3 vacias, el resto tambien)
filtro1 = pesajes_2015['VIDRIO'].isin([0,np.nan," "])
filtro2 = pesajes_2015['PAPEL_Y_CARTON'].isin([0,np.nan," "])
sin_vacio = (filtro1 & filtro2)
pesajes_2015_semanas_vacias = pesajes_2015[sin_vacio].drop_duplicates().sort_values(by=['PUNTO', 'MES','SEMANA']).copy().reset_index(drop=True)

filtro1 = pesajes_2016['VIDRIO'].isin([0,np.nan," "])
filtro2 = pesajes_2016['PAPEL_Y_CARTON'].isin([0,np.nan," "])
sin_vacio = (filtro1 & filtro2)
pesajes_2016_semanas_vacias = pesajes_2016[sin_vacio].drop_duplicates().sort_values(by=['PUNTO', 'MES','SEMANA']).copy().reset_index(drop=True)

In [17]:
# Leo todas las semanas faltantes, si llego a 3 en el mismo mes lo guardo en el
# siguiente dataframe para luego borrarlos todos juntos

meses_Vacios_2015 = pd.DataFrame({"PUNTO":[],"ANIO":[],"MES":[]})
mesAnterior = 0 

for indice in range(0, len(pesajes_2015_semanas_vacias)):
    mesActual = pesajes_2015_semanas_vacias.iloc[indice].MES
    if (mesAnterior == mesActual):
        semanas_vacias_del_Mes += 1
        if(semanas_vacias_del_Mes != 3):
            continue
        else:
            meses_Vacios_2015 = meses_Vacios_2015.append(
                {"PUNTO": pesajes_2015_semanas_vacias.iloc[indice].PUNTO,
                 "ANIO": 2015,
                 "MES": mesActual}, ignore_index=True)
    else:
        mesAnterior = mesActual
        semanas_vacias_del_Mes = 1 #Reseteo el contador

meses_Vacios_2015_tabla = meses_Vacios_2015.groupby(["PUNTO", "ANIO"])["MES"].count().sort_values(ascending=False)
meses_Vacios_2015_tabla = pd.concat([meses_Vacios_2015_tabla], axis=1, keys=['Meses vacios por tener al menos 3 semanas vacias'])

# Mismo pasa para 2016
meses_Vacios_2016 = pd.DataFrame({"PUNTO":[],"ANIO":[],"MES":[]})
mesAnterior = 0

for indice in range(0, len(pesajes_2016_semanas_vacias)):
    mesActual = pesajes_2016_semanas_vacias.iloc[indice].MES
    if (mesAnterior == mesActual):
        semanas_vacias_del_Mes += 1
        if(semanas_vacias_del_Mes != 3):
            continue
        else:
            meses_Vacios_2016 = meses_Vacios_2016.append(
                {"PUNTO": pesajes_2016_semanas_vacias.iloc[indice].PUNTO,
                 "ANIO": 2016,
                 "MES": mesActual}, ignore_index=True)
    else:
        mesAnterior = mesActual
        semanas_vacias_del_Mes = 1 #Reseteo el contador

meses_Vacios_2016_tabla = meses_Vacios_2016.groupby(["PUNTO", "ANIO"])["MES"].count().sort_values(ascending=False)
meses_Vacios_2016_tabla = pd.concat([meses_Vacios_2016_tabla], axis=1, keys=['Meses vacios por tener al menos 3 semanas vacias'])

2015 no tiene meses faltantes no cargados.

En el Año 2016 hay muchos meses que no estan cargados. Vamos a contarlos incluyendo los que estan vacíos por cada uno de los Puntos.

Luego eliminamos los barrios que tengan 4 o mas meses vacios entre los 2 años.

### Junto todas las tablas en una que muestre el estado de los Puntos segun datos vacios y faltantes

In [19]:
## Creo un df de los meses que tengo cargados de cada año, ordenados
meses_cargados_2015 = pesajes_2015.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
meses_cargados_2015 = meses_cargados_2015.copy().reset_index(drop=True)
meses_cargados_2016 = pesajes_2016.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
meses_cargados_2016 = meses_cargados_2016.copy().reset_index(drop=True)
    
## Los comparo y obtengo un df de todos los meses faltantes con PUNTO y MES
meses_faltantes = pd.merge(meses_cargados_2015, meses_cargados_2016,how='outer', indicator=True).query('_merge == "left_only"')
meses_faltantes_por_barrio = meses_faltantes.groupby(["PUNTO"])["MES"].count().sort_values(ascending=False)
    
## Agrego los 2 puntos sin falta de meses para que quede completa la lista
puntos_sin_meses_faltantes = list(filter(lambda x: x not in meses_faltantes_por_barrio.index, puntos_distintos))
meses_faltantes_por_barrio[puntos_sin_meses_faltantes[0]] = 0
meses_faltantes_por_barrio[puntos_sin_meses_faltantes[1]] = 0

Tabla_Faltantes = pd.concat([meses_faltantes_por_barrio], axis=1, keys=['Meses faltantes por barrio en 2016'])
meses_faltantes = meses_faltantes.iloc[:,0:2]

Tabla_de_Vacios_Completa = meses_faltantes.groupby(["PUNTO"])["MES"].count().sort_values(ascending=False)
## Agrego los 2 puntos sin falta de meses para que quede completa la lista
puntos_sin_meses_faltantes_2 = list(filter(lambda x: x not in Tabla_de_Vacios_Completa.index, puntos_distintos))
Tabla_de_Vacios_Completa[puntos_sin_meses_faltantes_2[0]] = 0;Tabla_de_Vacios_Completa[puntos_sin_meses_faltantes_2[1]] = 0
Tabla_de_Vacios_Completa = Tabla_de_Vacios_Completa.to_frame().rename(columns = {"MES":"Faltantes_2016"})


Agregar_a_tabla_vacias = pesajes_2016_semanas_vacias.groupby(["PUNTO", "MES"]).filter(lambda x: x["SEMANA"].count() >= 3)
Agregar_a_tabla_vacias_filtrada_2016 = Agregar_a_tabla_vacias.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
Agregar_a_tabla_vacias_filtrada_2016 = Agregar_a_tabla_vacias_filtrada_2016.groupby(["PUNTO"]).size().to_frame().rename(columns = {0: "Vacios_2016"})

Agregar_a_tabla_vacias_2015 = pesajes_2015_semanas_vacias.groupby(["PUNTO", "MES"]).filter(lambda x: x["SEMANA"].count() >= 3)
Agregar_a_tabla_vacias_filtrada_2015 = Agregar_a_tabla_vacias_2015.iloc[:,[0,3]].drop_duplicates().sort_values(by=['PUNTO', 'MES'])
Agregar_a_tabla_vacias_filtrada_2015 = Agregar_a_tabla_vacias_filtrada_2015.groupby(["PUNTO"]).size().to_frame().rename(columns = {0: "Vacios_2015"})

Tabla_de_Vacios_Completa = pd.merge(Tabla_de_Vacios_Completa, Agregar_a_tabla_vacias_filtrada_2016, how='left', left_index=True, right_index=True).fillna(0)
Tabla_de_Vacios_Completa = pd.merge(Tabla_de_Vacios_Completa, Agregar_a_tabla_vacias_filtrada_2015, how='left', left_index=True, right_index=True).fillna(0)
Tabla_de_Vacios_Completa['Meses_No_Utilizables'] = Tabla_de_Vacios_Completa.Faltantes_2016 + Tabla_de_Vacios_Completa.Vacios_2016 + Tabla_de_Vacios_Completa.Vacios_2015

### Decidimos borrar estos 4 Puntos por no cumplir el criterio de meses Vacios que definimos.

In [20]:
pesajes_criterio_de_limpieza = pesajes.loc[~pesajes['PUNTO'].isin(['ESTACION VILLA SOLDATI','PLAZA 25 DE AGOSTO','PLAZA VELAZCO IBARRA','PLAZA SAN MARTIN'])]

# Muestro la tabla ordenada por Meses No utilizables. Solo muestro los que no cumplen el criterio de meses faltantes
Tabla_de_Vacios_Completa.sort_values(by=['Meses_No_Utilizables'],ascending=False).iloc[0:4,:]

,Faltantes_2016,Vacios_2016,Vacios_2015,Meses_No_Utilizables
PUNTO,,,,
PLAZA 25 DE AGOSTO,4,8.0,11.0,23.0
ESTACION VILLA SOLDATI,4,8.0,4.0,16.0
PLAZA VELAZCO IBARRA,3,0.0,4.0,7.0
PLAZA M BELGRANO,3,2.0,0.0,5.0


### Observando el Dataset pudimos ver que existen muchos nulos en ciertas quintas semanas de 2015 y 2016, estas semanas al corroborar con el calendario tenian muy pocos días. Asuminos esta como la razon de porque no se cargaron en casi ninguna comuna y las eliminamos

Podemos ver el estado actual de nulos para compararlo cuando eliminemos las quintas semanas del mes 6 y 11 para 2015 y el mes 1 y 9 para 2016

In [21]:
pesajes_con_nulo = pesajes_criterio_de_limpieza.replace('', np.nan).replace(' ', np.nan)
pesajes_1 = pesajes_con_nulo.loc[~((pesajes_con_nulo['ANIO'] == 2015) & (pesajes_con_nulo['MES'] == 6) & (pesajes_con_nulo['SEMANA'] == 5))]
pesajes_2 = pesajes_1.loc[~((pesajes_1['ANIO'] == 2015) & (pesajes_1['MES'] == 11) & (pesajes_1['SEMANA'] == 5))]
pesajes_3 = pesajes_2.loc[~((pesajes_2['ANIO'] == 2016) & (pesajes_2['MES'] == 1) & (pesajes_2['SEMANA'] == 5))]
pesajes_sin_quinta_vacias = pesajes_3.loc[~((pesajes_3['ANIO'] == 2016) & (pesajes_3['MES'] == 9) & (pesajes_3['SEMANA'] == 5))]

pesajes_ordenado = pesajes_sin_quinta_vacias.sort_values(by=['ANIO','MES'])
print("Nulos restantes"); pesajes_ordenado.isnull().sum()[pesajes_ordenado.isnull().sum() > 0]

Nulos restantes


VIDRIO                        38
PAPEL_Y_CARTON                39
METAL                         39
TELGOPOR                      39
PLASTICOS_TODOS_LOS_DEMAS     37
TETRA_BRICK                  377
dtype: int64

### Como termina en la tercera semana de Septiembre de 2016, la cuarta esta vacia. Decidimos llenar esta cuarta semana con una media da las primeras 3 semanas

In [22]:
for indice in range(0, len(pesajes_ordenado)):
    if ((pesajes_ordenado.iloc[indice,2:5] == [2016, 9, 4]).all()):
        media = round((pesajes_ordenado.iloc[indice-3,5:12].apply(lambda x: float(x)) +
                 pesajes_ordenado.iloc[indice-2,5:12].apply(lambda x: float(x)) +
                 pesajes_ordenado.iloc[indice-1,5:12].apply(lambda x: float(x)))
                 /3, 3)
        pesajes_ordenado.iloc[indice,5:12] = media.apply(lambda x: str(x))
        
#Relleno a mano 5 excepciones con las medias de los meses cercanos a ellas
for i in range (0,5):
    pesajes_ordenado.iloc[1996+i,5:12] = pesajes_ordenado.iloc[1967,5:12]
    
pesajes_ordenado.loc[2038,'PLASTICOS_TODOS_LOS_DEMAS'] = pesajes_ordenado.loc[2037,'PLASTICOS_TODOS_LOS_DEMAS']
print("Nulos restantes"); pesajes_ordenado.isnull().sum()[pesajes_ordenado.isnull().sum() > 0]

Nulos restantes


VIDRIO                        12
PAPEL_Y_CARTON                13
METAL                         13
TELGOPOR                      13
PLASTICOS_TODOS_LOS_DEMAS     10
TETRA_BRICK                  351
dtype: int64

### El resto de nulos (no tetra_brick) que quedan los rellenamos haciendo un promedio de sus semanas llenas en ese mes.

In [23]:
pesajes_ordenado = pesajes_ordenado.replace('nan', np.nan)
VidriosNulos = pesajes_ordenado[pesajes_ordenado.iloc[:,5].isnull()] #VIDRIOS
PyCNulos = pesajes_ordenado[pesajes_ordenado.iloc[:,6].isnull()]     #PAPEL Y CARTON
MetalNulos = pesajes_ordenado[pesajes_ordenado.iloc[:,7].isnull()]   #METAL
TelgoporNulos = pesajes_ordenado[pesajes_ordenado.iloc[:,8].isnull()]#TELGOPOR
matsNulos = [VidriosNulos, PyCNulos, MetalNulos, TelgoporNulos]
material = ['VIDRIO','PAPEL_Y_CARTON','METAL','TELGOPOR']

# Relleno con la media de las semanas restantes del mes con algunas nulas mas las del siguiente y el anterior
for m in range(0, 4):
    for i in range(0, len(matsNulos[m])):
        mesActual = matsNulos[m]['MES'].iloc[i]
        medias = (pesajes_ordenado.loc[(pesajes_ordenado['MES'].isin([mesActual, mesActual-1, mesActual+1])) & (pesajes_ordenado['ANIO'] == matsNulos[m]['ANIO'].iloc[i]) & (pesajes_ordenado['PUNTO'] == matsNulos[m]['PUNTO'].iloc[i]) & 
                                       (pesajes_ordenado[material[m]].notnull())].iloc[:,[5,6,7,9,10]]).apply(lambda x: x.apply(lambda x: float(x))).mean().apply(lambda x: str(round(x,2)))
        count = 0
        for c in ['VIDRIO','PAPEL_Y_CARTON','METAL','TELGOPOR','PLASTICOS_TODOS_LOS_DEMAS']:
            pesajes_ordenado.loc[matsNulos[m].index[i], c] = medias[count]
            count+=1

print("Nulos restantes"); pesajes_ordenado.isnull().sum()[pesajes_ordenado.isnull().sum() > 0]

Nulos restantes


TETRA_BRICK    352
dtype: int64

### Por ultimo llenamos los Tetra Bricks que faltan usando las medias de las semanas(o mes en peor caso) anterior y siguiente

In [24]:
# En meses 1,2 y 3 de 2015 estan casi todos los vacios, los relleno con la media en el mismo punto de los meses 4 y 5
EneFebMar_TetraNulos = pesajes_ordenado.loc[(pesajes_ordenado['ANIO'] == 2015) & (pesajes_ordenado['MES'].isin([1,2,3])) & (pesajes_ordenado['TETRA_BRICK'].isnull())]

# Relleno con la media del los meses 4 y 5 que sean de mismo Punto
for i in range(0, len(EneFebMar_TetraNulos)):
    media = (pesajes_ordenado.loc[(pesajes_ordenado['MES'].isin([4,5])) & (pesajes_ordenado['ANIO'] == 2016) & (pesajes_ordenado['PUNTO'] == EneFebMar_TetraNulos['PUNTO'].iloc[i]) & 
                              (pesajes_ordenado['TETRA_BRICK'].notnull())].iloc[:,10]).apply(lambda x: float(x)).mean()
    media = str(round(media,2))
    pesajes_ordenado.loc[EneFebMar_TetraNulos.index[i],'TETRA_BRICK'] = media

# Los pocos Nulos faltantes los lleno las semanas con las medias de sus respectivos meses en sus respectivos puntos
TetraUltimosNulos = pesajes_ordenado[pesajes_ordenado.iloc[:,10].isnull()]
for i in range(0, len(TetraUltimosNulos)):
    mesActual = TetraUltimosNulos['MES'].iloc[i]
    media = (pesajes_ordenado.loc[(pesajes_ordenado['MES'].isin([mesActual, mesActual-1, mesActual+1])) & (pesajes_ordenado['ANIO'] == TetraUltimosNulos['ANIO'].iloc[i]) & (pesajes_ordenado['PUNTO'] == TetraUltimosNulos['PUNTO'].iloc[i]) & 
                                  (pesajes_ordenado['TETRA_BRICK'].notnull())].iloc[:,10]).apply(lambda x: float(x)).mean()
    media = str(round(media,2))
    pesajes_ordenado.loc[TetraUltimosNulos.index[i],'TETRA_BRICK'] = media

print(f"Chequeamos si quedo algun pasaje nulo: {pesajes_ordenado.isnull().sum().any()}")

Chequeamos si quedo algun pasaje nulo: False


In [25]:
export_csv = pesajes_ordenado.to_csv(r'pesajes_nulos.csv', index = None, header=True, sep=';')